# Train Model

This notebook is responsible for training the image classification model.
It will load the processed data, define the model architecture, set up the training loop, and save the trained model and relevant artifacts.

In [3]:
import sys
import os
from pathlib import Path
import importlib
from IPython.display import Image

# Add the project root to the Python path
# This allows importing modules from the 'src' directory
current_path = Path(os.getcwd()).resolve()
project_root = None
# Iterate up from current_path to its parents
for parent_dir in [current_path] + list(current_path.parents):
    if (
        (parent_dir / ".git").is_dir()
        or (parent_dir / "pyproject.toml").is_file()
        or (parent_dir / "src").is_dir()
    ):
        project_root = parent_dir
        break

if project_root is None:
    # Fallback for structures where notebook is in 'notebooks' dir directly under project root
    if current_path.name == "notebooks" and (current_path.parent / "src").is_dir():
        project_root = current_path.parent
    else:
        # Default to current_path if specific markers or 'notebooks' structure isn't found
        project_root = current_path
        print(
            f"Warning: Could not reliably find project root. Using CWD: {project_root}. Ensure 'src' is in python path."
        )

if project_root:
    project_root_str = str(project_root)
    if project_root_str not in sys.path:
        sys.path.insert(0, project_root_str)
        print(f"Project root '{project_root_str}' added to sys.path.")
    else:
        print(f"Project root '{project_root_str}' is already in sys.path.")
else:
    print("Error: Project root could not be determined. Imports from 'src' may fail.")

# Reload modules to ensure the latest changes are picked up
# Useful if you're actively developing the src modules
import src.config
import src.data.loader
import src.models.PhotoTagNet_model
import src.models.basic_model
import src.utils.seed
import src.utils.plot

importlib.reload(src.config)
importlib.reload(src.data.loader)
importlib.reload(src.models.PhotoTagNet_model)
importlib.reload(src.models.basic_model)
importlib.reload(src.utils.seed)
importlib.reload(src.utils.plot)
from sympy import Basic
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm  # For progress bars

from src.models.PhotoTagNet_model import PhotoTagNet

# Imports from our src directory
from src.config import (
    ModelConfig,
    OptimConfig,
    TrainConfig,
    CHECKPOINT_DIR,
    RESULTS_DIR,
)
from src.config import DEFAULT_CLASSES
from src.data.loader import load_data
from src.models.basic_model import BasicMLC
from src.utils.seed import set_seed
from src.utils.plot import save_loss_plot

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Project root '/workspaces/photo_tag_pipeline' is already in sys.path.
Using device: cpu


In [4]:
# Ensure results and plots directories exist for storing outputs
PLOTS_DIR = RESULTS_DIR / "plots"
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
PLOTS_DIR.mkdir(exist_ok=True, parents=True)

# ---- Configurations ----
mcfg = ModelConfig()
ocfg = OptimConfig()
tcfg = TrainConfig()

print(f"TrainConfig: {tcfg}")
print(f"ModelConfig: {mcfg}")
print(f"OptimConfig: {ocfg}")

# ---- Set Seed ----
set_seed(tcfg.seed)
print(f"Seed set to {tcfg.seed}")

# ---- Data Loaders ----
print("Loading data...")
train_dataset, val_dataset, train_loader, val_loader = load_data()
print(
    f"Data loaded. Train batches: {len(train_loader)}, Val batches: {len(val_loader)}"
)


# ---- Model, Loss, Optimizer ----
print("Building model...")
model = BasicMLC(len(DEFAULT_CLASSES)).to(DEVICE)
# model = PhotoTagNet(ModelConfig(), num_classes=len(DEFAULT_CLASSES)).to(DEVICE)
criterion = (
    nn.BCEWithLogitsLoss()
)  # Binary Cross-Entropy for multi-label with sigmoid output
optimizer = optim.AdamW(model.parameters(), lr=ocfg.lr, weight_decay=ocfg.weight_decay)
print("Model, criterion, and optimizer created.")


# ---- Training Loop ----
best_val_loss = float("inf")
train_losses, val_losses = [], []

print(f"Starting training for {tcfg.epochs} epochs...")
for epoch in range(tcfg.epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(
        train_loader, desc=f"Epoch {epoch+1}/{tcfg.epochs} [Training]", unit="batch"
    )
    for imgs, labels in progress_bar:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # ---- Validation ----
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        progress_bar_val = tqdm(
            val_loader, desc=f"Epoch {epoch+1}/{tcfg.epochs} [Validation]", unit="batch"
        )
        for imgs, labels in progress_bar_val:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            progress_bar_val.set_postfix(loss=loss.item())

    val_loss = val_running_loss / len(val_loader)
    val_losses.append(val_loss)

    print(
        f"Epoch {epoch+1}/{tcfg.epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}"
    )

    # ---- Checkpoint ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_path = CHECKPOINT_DIR / "best_model_notebook.pth"
        torch.save(model.state_dict(), best_model_path)
        print(
            f"New best model saved to {best_model_path} (Val Loss: {best_val_loss:.4f})"
        )

# ---- Save Final Model ----
final_model_path = CHECKPOINT_DIR / "final_model_notebook.pth"
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to {final_model_path}")

# ---- Plot and Save Loss Curve ----
# Ensure results directory exists (though config should handle it)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
loss_plot_path = save_loss_plot(
    train_losses, val_losses, str("loss_curve_notebook.png")
)
print(f"Loss curve saved to {loss_plot_path}")

print("Training complete.")

# Display the plot in the notebook
display(Image(filename=str(loss_plot_path)))

TrainConfig: TrainConfig(epochs=30, seed=42, precision_at_k=5, early_stop_patience=7)
ModelConfig: ModelConfig(backbone='resnet50', pretrained=True, freeze_backbone=False, dropout_rate=0.3)
OptimConfig: OptimConfig(optim='adamw', lr=0.0003, weight_decay=0.0001, betas=(0.9, 0.999), momentum=0.9, scheduler='step', step_size=5, gamma=0.5, patience=5)
Seed set to 42
Loading data...
Data loaded. Train batches: 39, Val batches: 6
Building model...
Model, criterion, and optimizer created.
Starting training for 30 epochs...


Epoch 1/30 [Validation]: 100%|██████████| 6/6 [00:01<00:00,  5.14batch/s, loss=0.14] 


Epoch 1/30 - Train Loss: 0.3938, Val Loss: 0.2401
New best model saved to /workspaces/photo_tag_pipeline/checkpoints/best_model_notebook.pth (Val Loss: 0.2401)


Epoch 2/30 [Validation]: 100%|██████████| 6/6 [00:01<00:00,  5.03batch/s, loss=0.142]


Epoch 2/30 - Train Loss: 0.2235, Val Loss: 0.3014


...

Epoch 12/30 - Train Loss: 0.0484, Val Loss: 0.3797


Epoch 13/30 [Validation]: 100%|██████████| 6/6 [00:01<00:00,  3.97batch/s, loss=0.222]


Epoch 13/30 - Train Loss: 0.0420, Val Loss: 0.3704


Epoch 14/30 [Training]:  67%|██████▋   | 26/39 [00:18<00:09,  1.38batch/s, loss=0.0367] 


KeyboardInterrupt: 

In [ ]:
import subprocess
import mlflow
import os

# Check if mlflow is installed and start the UI if available
try:
    mlflow_port = 5000
    mlflow_ui_proc = subprocess.Popen(
        ["mlflow", "ui", "--port", str(mlflow_port), "--host", "0.0.0.0"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    print(f"MLflow UI started on port {mlflow_port}.")
    # Open in host browser if $BROWSER is available
    if "BROWSER" in os.environ:
        os.system(f"$BROWSER http://localhost:{mlflow_port}")
    else:
        print(f"Open http://localhost:{mlflow_port} in your browser.")
except ImportError:
    print("mlflow is not installed. Please install it with `pip install mlflow`.")

After training, the model will be saved to the `checkpoints/` directory (e.g., `best_model_notebook.pth`, `final_model_notebook.pth`), and the loss curve plot will be saved in the `results/` directory (e.g., `loss_curve_notebook.png`).

In [ ]:
import os
from IPython.display import HTML
from IPython import get_ipython

# Get current notebook name
try:
    # Try to get the notebook name using IPython's special variable
    notebook_path = (
        get_ipython().kernel.shell.user_ns["__vsc_ipynb_file__"]
        if "__vsc_ipynb_file__" in get_ipython().kernel.shell.user_ns
        else None
    )
    if not notebook_path:
        notebook_path = get_ipython().kernel.shell.user_ns.get(
            "__notebook_source__", ""
        )
    notebook_name = os.path.basename(notebook_path) if notebook_path else ""
except:
    notebook_name = ""

# Check if the notebook name was successfully determined
if notebook_name == "":
    # Fallback method if automatic detection fails
    notebook_name = input("Enter notebook filename (with .ipynb extension): ")

# Use nbconvert to export the notebook without input cells tagged with "remove"
# Ensure the assets directory exists
assets_dir = "../assets"
os.makedirs(assets_dir, exist_ok=True)
output_html = os.path.join(
    assets_dir, os.path.splitext(os.path.basename(notebook_name))[0] + "_export.html"
)
!jupyter nbconvert --to html --TagRemovePreprocessor.remove_cell_tags='{"remove"}' "{notebook_name}" --output "{output_html}"

# Display a success message
display(
    HTML(
        f"<div style=padding:10px;'>"
        f"<h3>Export complete!</h3>"
        f"<p>Notebook <b>{notebook_name}</b> has been exported to HTML.</p>"
        f"</div>"
    )
)